# <center>Towards control of opinion diversity by introducing zealots into a polarised social group </center>
Code used in the paper submitted to Discovery Science 2021.

In [ ]:
# imports
import sys
import numpy as np
import pickle
from time import time
import matplotlib.pyplot as plt
from matplotlib import rc
import networkx as nx
from matplotlib.lines import Line2D
import util # our functions

# for plots appearance
color = ["blue", "red", "green", "orange"]
marker = "x+*^p"
linestyle = ["--", "-.", ":", "-", (0,(3, 5, 1, 5))]

# latex rendering
rc('font',**{'family':'sans-serif','sans-serif':['Palatino']})
rc('text', usetex=True)
rc('legend', frameon=False) # PS doesn't do well with transparencies

## Figure 1.
Choose the parameters.

In [ ]:
n = 50 # nb users
n1 = 10 # N1(t=0)
z0, z1 = 5, 2  # number of zealots
max_time = 20
spacing_simu = 1
length = int(np.floor(max_time/spacing_simu)+1)

Simulation.

In [ ]:
N1t = util.voter_simu(n, n1, z1, z0, max_time, spacing_simu)
N1t = N1t.astype(int)

Plot.

In [ ]:
fs = 10 # fontsize
fig, ax = plt.subplots(1,4, figsize=(7,2))
plot_times = (0,5,10,12)
        
# create graph
G = nx.Graph()
for k in range(n):
    if k<z0 or k>=n-z1:
        G.add_node(k, s="^")
    else:
        G.add_node(k, s="o")
    for l in range(k):
        G.add_edge(l,k)

# get node pos and all distinct node classes according to the node shape attribute
pos = nx.layout.spring_layout(G, scale=1)
nodeShapes = set((aShape[1]["s"] for aShape in G.nodes(data=True)))

# look at different times
for i,t in enumerate(plot_times):
    n0t, n1t = n-N1t[t]-z0, N1t[t]-z1
    color = ["red"]*z0 + ["darkred"]*n0t + ["blue"]*n1t + ["deepskyblue"]*z1

    # draw nodes
    for aShape in nodeShapes:
        node_list = [sNode[0] for sNode in filter(lambda x: x[1]["s"]==aShape, G.nodes(data=True))]
        color_list = [color[k] for k in node_list]
        nx.draw_networkx_nodes(G, pos, ax=ax[i], node_size=250, node_color=color_list, node_shape=aShape, nodelist=node_list, edgecolors="black")
        ax[i].axis("off")
        ax[i].set_title("t={}".format(t))
        
    # draw edges if needed
    #nx.draw_networkx_edges(G, pos, ax=axis, width=.1)

# legend
legend_elements = list()
legend_elements += [Line2D([0], [0], marker="^", color="w", label="0-zealot", markerfacecolor="red", markeredgecolor="black", markersize=15)]
if z1>0:
    legend_elements += [Line2D([0], [0], marker="^", color="w", label="1-zealot", markerfacecolor="deepskyblue", markeredgecolor="black", markersize=15)]
legend_elements += [Line2D([0], [0], marker="o", color="w", label="opinion-0", markerfacecolor="darkred", markeredgecolor="black", markersize=15)]
legend_elements += [Line2D([0], [0], marker="o", color="w", label="opinion-1", markerfacecolor="blue", markeredgecolor="black", markersize=15)]
if z1>0:
    plt.legend(handles=legend_elements, edgecolor="w", labelspacing=.6, fontsize=fs, handletextpad=.25, borderpad=.2, loc=(1.2,0.18))
else:
    plt.legend(handles=legend_elements, edgecolor="w", labelspacing=.6, fontsize=fs, handletextpad=.25, borderpad=.2, loc=(1.2,0.27))

# show
plt.tight_layout()
plt.show()
plt.close()

## Figures 2 and 3
Choose parameters. Erdos-Renyi of parameter 1 = complete graph.

In [ ]:
n = 100
z0, z1 = 20, 40 # nb of zealots
n_simu = 500 # nb of simulations
max_time = 200 # max time for each simulation
spacing_simu = .1 # save N1t every .1 time units
length = int(np.floor(max_time/spacing_simu)+1) # don't change
x_axis = np.linspace(0, max_time, length) # don't change
graph_model = ("ER","WS","BA") # chosen graph models
model_name = {"ER":"Erdös-Rényi", "WS":"Watts-Strogatz", "BA":"Barabási-Albert"} # full names
graph_param = {"ER":(.1,.3,.5,1), "BA":(1,3,5), "WS":(0.01,.05,0.1)} # parameters for each graph model
n_param = {model: len(graph_param[model]) for model in graph_model} # don't change
param_name = {"ER":"p", "BA":r"m", "WS":r"$\omega$"} # don't change
color = ["blue", "red", "green", "orange"] # graph colors

Compute expected limiting average opinion.

In [ ]:
n1_equilibrium = n*z1/(z0+z1)

Simulate for each model and parameter.

In [ ]:
N1t = {model: np.zeros((n_simu, length, n_param[model])) for model in graph_model}
start = time()

for model in graph_model:
     for i,param in enumerate(graph_param[model]):
        leaders = util.create_connected_user_graph(n,model,param)
        for k in range(n_simu): #progressbar.progressbar(range(n_simu)):
            n1 = np.random.randint(z1,n-z0+1)
            sys.stdout.flush()
            sys.stdout.write("Graph {}. Param {}/{}. Simu {}/{}. Elapsed time {:.3f}\r".format(model, i+1, n_param[model], k+1, n_simu, time()-start))
            N1t[model][k,:,i] = util.custom_graph_simu(leaders, n1, z1, z0, max_time, spacing_simu)

Plot figure 2.

In [ ]:
# choose custom time range to plot
max_time_plot = 50
time_range = x_axis<=max_time_plot

fig, ax = plt.subplots(2, 2, figsize=(7,4))

# complete first
mean = N1t["ER"][:,time_range,-1].mean(axis=0)
ax[0,0].axhline(y=n1_equilibrium, color="grey", label=r"$nz_1/(z_0+z_1)$")
ax[0,0].plot(x_axis[time_range], mean, c=color[0])
ax[0,0].set_title("Complete")
ax[0,0].legend(loc="best", frameon=True, fontsize=9, edgecolor="grey")

# then the rest
for j,model in enumerate(graph_model,1):
    x,y = j//2,j%2 # axis index
    ax[x,y].axhline(y=n1_equilibrium, color="grey", label=r"$nz_1/(z_0+z_1)$")
    for i,param in enumerate(graph_param[model]):
        if model=="ER" and param==1: # we already plotted complete
            continue
        else:
            mean = N1t[model][:,time_range,i].mean(axis=0)
            ax[x,y].plot(x_axis[time_range], mean, ls=linestyle[i], c=color[i], label=r"{}={}".format(param_name[model],param))

    # style
    ax[x,y].legend(loc="best", frameon=True, fontsize=9, edgecolor="grey")
    ax[x,y].set_title(model_name[model])
        

# style, show and save
ax[1,0].set_xlabel(r"time")
ax[1,0].set_ylabel(r"$N_1(t)$")
plt.tight_layout()
plt.show()
plt.close()

Plot figure 3.

In [ ]:
cv_time = 20 # estimated convergence time
time_range = x_axis>=cv_time

fig, ax = plt.subplots(2, 2, figsize=(7,4))

# complete first
unique, count = np.unique(N1t["ER"][:,time_range,-1], return_counts=True)
distrib = count/count.sum() # transform counts into proportions
ax[0,0].plot(unique, distrib, lw=.8, marker=marker[0], color=color[0])
ax[0,0].axvline(x=N1t["ER"][:,time_range,-1].mean(), lw=.8,  color=color[i])
ax[0,0].axvline(x=n1_equilibrium, lw=.8, color="grey")
ax[0,0].set_title("Complete")
ax[0,0].set_yticks([])

for j,model in enumerate(graph_model,1):
    x,y = j//2,j%2 # axis index
    #ax[x,y].plot(range(z1, n-z0+1), stationary, color="grey", lw=.8, label=r"$nz_1/(z_0+z_1)$")
    ax[x,y].axvline(x=n1_equilibrium, lw=.8, color="grey")
    for i,param in enumerate(graph_param[model]):
        if model=="ER" and param==1:
            continue
        else:
            unique, count = np.unique(N1t[model][:,time_range,i], return_counts=True)
            distrib = count/count.sum() # transform counts into proportions
            ax[x,y].plot(unique, distrib, linewidth=.8, ls=linestyle[i], marker=marker[i], color=color[i], label=r"{}={}".format(param_name[model],param))
            ax[x,y].axvline(x=N1t[model][:,time_range,i].mean(), lw=.8, color=color[i])
    
    ax[x,y].set_title(model_name[model])
    ax[x,y].legend(loc="upper left", frameon=True)
    ax[x,y].set_yticks([])
    
# end
ax[1,0].set_xlabel(r"$N_1^\star$")
plt.tight_layout()
plt.show()
plt.close()

Compute averaged relative errors.

In [ ]:
cv_time = 20
time_range = x_axis>=cv_time
for model in graph_model:
    for i,param in enumerate(graph_param[model]):
        temporal_mean = N1t[model][:,time_range,i].mean(axis=1)
        error = np.abs(temporal_mean - n1_equilibrium)/n1_equilibrium
        error_avg = error.mean()
        std_avg = error.std()
        print("[{} {}]".format(model,param), error_avg, std_avg)

Compute averaged absolute errors.

In [ ]:
cv_time = 20
time_range = x_axis>=cv_time
for model in graph_model:
    for i,param in enumerate(graph_param[model]):
        temporal_mean = N1t[model][:,time_range,i].mean(axis=1)
        error = np.abs(temporal_mean - n1_equilibrium)
        error_avg = error.mean()
        std_avg = error.std()
        print("[{} {}]".format(model,param), error_avg, std_avg)

## Figures 4 and 5
Choose parameters. We consider several different values for $\lambda,\alpha,z_0$.

In [ ]:
n = 100
lambda_range = np.array([0.3, 0.5, 0.7]) # heterogeneity
z0_range = np.arange(n)
alpha_range = np.array([.025, .05, 0.075, .1]) # backfire effect intensity
with_max = True # do we use a z1_max?

Compute optimum $z_1^\star$ for each set of parameters.

In [ ]:
Z1 = np.zeros((alpha_range.size, z0_range.size, lambda_range.size))
for k,lambd in enumerate(lambda_range):
    for i,alpha in enumerate(alpha_range):
        for j,z0 in enumerate(z0_range):
            D = 1-lambd-lambd*alpha*z0
            if D <= 0:
                Z1[i,j,k] = -1
            else:
                z1 = lambd*z0/D
                if with_max:
                    z1_max = (n-z0)/(1+alpha*z0)
                    Z1[i,j,k] = min(z1_max, z1)
                else:
                    Z1[i,j,k] = z1

Plot Figure 4.

In [ ]:
fs = 15 # fontsize
fig, ax = plt.subplots(1, 3, figsize=(12,3))

for k,lambd in enumerate(lambda_range):
    for i,alpha in enumerate(alpha_range):
        idx, = np.where(Z1[i,:,k]!=-1) # don't plot when D<=0
        ax[k].plot(z0_range[idx], Z1[i,idx,k], marker=marker[i], ls=linestyle[i], color=color[i], markevery=2, label=r"$\alpha={}$".format(alpha))
    ax[k].grid()
    ax[k].set_title(r"$\lambda={}$".format(lambd), fontsize=fs)
    
# style
ax[0].set_ylabel(r"$z_1^\star$", fontsize=fs)
ax[1].set_xlabel(r"$z_0$", fontsize=fs)
ax[2].legend(loc="lower right", frameon=True, framealpha=1)
    
# save and show
plt.tight_layout()
plt.show()
plt.close()

Now we look at the discrepancy between the resulting diversity and lambda in the $D\le0$ cases.

In [ ]:
error = np.zeros((alpha_range.size, z0_range.size, lambda_range.size))
for k,lambd in enumerate(lambda_range):
    for i,alpha in enumerate(alpha_range):
        for j,z0 in enumerate(z0_range):
            D = 1-lambd-lambd*alpha*z0
            if D <= 0:
                zm = (n-z0)/(1+alpha*z0) # z1_max
                error[i,j,k] = np.abs(lambd - zm/((1+alpha*zm)*z0+zm))
            else:
                continue # error is already 0 from the way we initialised

Plot Figure 5.

In [ ]:
fs = 15 # fontsize
fig, ax = plt.subplots(1, 3, figsize=(12,3))

for k,lambd in enumerate(lambda_range):
    for i,alpha in enumerate(alpha_range):
        idx, = np.where(Z1[i,:,k]==-1)
        ax[k].plot(z0_range[idx], error[i,idx,k], marker=marker[i], markevery=5, lw=.8, ls=linestyle[i], color=color[i], label=r"$\alpha={}$".format(alpha))
    ax[k].grid()
    ax[k].set_title(r"$\lambda={}$".format(lambd), fontsize=fs)
    
# style
ax[0].set_ylabel(r"error", fontsize=fs)
ax[1].set_xlabel(r"$z_0$", fontsize=fs)
ax[2].legend(loc="lower right", frameon=True, framealpha=1)
    
# save and show
plt.tight_layout()
plt.show()
plt.close()